## Example Usage of the GMail Python API

This Notebook shows the basic usage of the GMail API

1. Load E-Mails from the server

2. Send E-Mails with the server

In order to be able to use the API you need a valid token as a .pickle file. In the first initialization of the GMail service you need to pass the path to this token.pickle file. Change the parameter below to your file to rerun the examples. 


For furhter information on how to send attachments, ... please consult the Documentation: 

https://googleapis.github.io/google-api-python-client/docs/epy/index.html

and the Guide: 

https://developers.google.com/gmail/api/quickstart/python

In [48]:
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
# !pip install typing

In [1]:
from googleapiclient.discovery import build, Resource
from email.mime.text import MIMEText
import base64
import pickle
import os
from typing import Dict
import email
import pandas as pd
import numpy as np
import torch
from transformers import *
from scipy.special import softmax

### Basic Functions

In [90]:
def load_service(token_path:str):
    """ Load the service from the specifies token file

    Args:
        token_path (str): Authorized Gmail API service instance.
    """
    if os.path.exists(token_path):
        with open(token_path, 'rb') as token:
            creds = pickle.load(token)
            service = build('gmail', 'v1', credentials=creds)
            return service

    else: 
        print("Credentials not found")
        return None

In [91]:
def send_message(service:Resource, user_id:str, message:Dict[str,str]):
    """Send an email message.

    Args:
        service (Resource): Authorized Gmail API service instance.
        user_id (str): User's email address. The special value "me" can be used to indicate the authenticated user.
        message (Dict[str,str]): Message to be sent.
    """
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                  .execute())
        print('Message Id: %s' % message['id'])
    except Exception as e:
        print('An error occurred: %s' % e)

# Create Message

In [162]:
def create_message(sender:str, to:str, subject:str, message_text:str) -> Dict[str,str]:

    message = MIMEText(message_text,"html",_charset="utf-8")
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    encoded_message = base64.urlsafe_b64encode(message.as_bytes())
    return {'raw': encoded_message.decode()}

In [161]:
message = create_message("datathon@lumos-consulting.at", "team4@lumos-consulting.at", "Test1","Tasersterö")
# message = create_message("team4@lumos-consulting.at", "team4@lumos-consulting.at", "Test_1", "Ich habe WAREHOUSE Produkte im IT&OFFICEKatalog15 firstclass standardmaessig installiert. Die Installation lief fehlerfrei ab und wurde ordnungsgemaess abgeschlossen. Das Programm wurde anschliessend registriert.  Beim Programmaufruf kommt fuer ca. 4-5 Sekunden die Sanduhr auf dem Bildschirm, sonst keine weitere Reaktionen, keine Fehlermeldungen, Warnungen etc. Das Programm wird nicht gestartet.")
# message

VGFzZXJzdGVyw7Y=



In [93]:
def receive_unread_messages(service:Resource):
    """ Recieve all unread messages and print them

        Args:
        service (Resource): Authorized Gmail API service instance.

    """
    messages = []
    # request a list of all the messages 
    result = service.users().messages().list(userId='me').execute() 
  
    # iterate through all the messages 
    for msg in result.get('messages') : 
        # Get the message from its id   
        # Use try-except to avoid any Errors 
        message = service.users().messages().get(userId="me", id=msg["id"],
                                             format='raw').execute()
        
        
        if "UNREAD" in message["labelIds"]:
            try:
                print(message["id"])
                msg_str = base64.urlsafe_b64decode(message['raw'].encode("utf-8")).decode("utf-8")
                print(msg_str)
                mime_msg = email.message_from_string(msg_str)
                
                messages.append((message["id"],mime_msg))
    
            except: 
                print("error")
                pass
    return messages

In [165]:
def receive_all_messages(service:Resource):
    """ Recieve all messages and print them

        Args:
        service (Resource): Authorized Gmail API service instance.

    """
    messages = []
    # request a list of all the messages 
    result = service.users().messages().list(userId='me').execute() 
  
    # iterate through all the messages 
    for msg in result.get('messages') : 
        # Get the message from its id   
        # Use try-except to avoid any Errors 
        message = service.users().messages().get(userId="me", id=msg["id"],
                                             format='raw').execute()
        
        
        
        try:
            print(message["id"])
            msg_str = base64.urlsafe_b64decode(message['raw'].encode("utf-8")).decode("utf-8")
            print(msg_str)
            mime_msg = email.message_from_string(msg_str)

            messages.append((message["id"],mime_msg))

        except: 
            print("error")
            pass
    return messages

In [95]:
def print_message(message):
    """ Print the Content of a Message

    Args:
        message (email): The email encoded message
    """
    try:
        if message.is_multipart():
            for payload in message.get_payload():
                print(payload.get_payload())
        else:
            print(message.get_payload())
    except:
        message.get_payload()

In [96]:
def get_text_from_message(message):
    """ Print the Content of a Message

    Args:
        message (email): The email encoded message
    """
    retText=""
    try:
        if message.is_multipart():

            for payload in message.get_payload():
                retText+=str(payload.get_payload())
                retText+="\n"
        else:
            retText+=str(message.get_payload())
    except:
        message.get_payload()
    return retText

### Eigene Funktionen

In [97]:
def markAsRead(service:Resource, message_id):
    if type(message_id) is list:
        for mid in message_id:
            service.users().messages().modify(userId='me', id=mid, body={'removeLabelIds': ['UNREAD']}).execute()
    else:
        service.users().messages().modify(userId='me', id=message_id, body={'removeLabelIds': ['UNREAD']}).execute()
    
def deleteMail(service:Resource, message_id):
    if type(message_id) is list:
        for mid in message_id:
            service.users().messages().delete(userId="me", id=mid).execute()
    else:
        service.users().messages().delete(userId="me", id=message_id).execute()

In [98]:
# Nötig, wenn Nachrichten von Google reinkommen.
# deleteMail(service,"1777346d2ae8e766")

### Usage of the function

In [99]:
token_path = "token.pickle"

In [100]:
service = load_service(token_path)

In [166]:
messages = receive_all_messages(service)

177737d99b836960
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 10:39:06 -0800
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64
to: team4@lumos-consulting.at
from: team4@lumos-consulting.at
subject: Test_1
Date: Fri, 5 Feb 2021 10:39:06 -0800
Message-Id: <CAEzfA9rqW56A3LSUJOu5KywkeH-G+tWL4sPoZaFsKqabDYL1nw@mail.gmail.com>

SWNoIGhhYmUgV0FSRUhPVVNFIFByb2R1a3RlIGltIElUJk9GRklDRUthdGFsb2cxNSBmaXJzdGNs
YXNzIHN0YW5kYXJkbWFlc3NpZyBpbnN0YWxsaWVydC4gRGllIEluc3RhbGxhdGlvbiBsaWVmIGZl
aGxlcmZyZWkgYWIgdW5kIHd1cmRlIG9yZG51bmdzZ2VtYWVzcyBhYmdlc2NobG9zc2VuLiBEYXMg
UHJvZ3JhbW0gd3VyZGUgYW5zY2hsaWVzc2VuZCByZWdpc3RyaWVydC4gIEJlaW0gUHJvZ3JhbW1h
dWZydWYga29tbXQgZnVlciBjYS4gNC01IFNla3VuZGVuIGRpZSBTYW5kdWhyIGF1ZiBkZW0gQmls
ZHNjaGlybSwgc29uc3Qga2VpbmUgd2VpdGVyZSBSZWFrdGlvbmVuLCBrZWluZSBGZWhsZXJtZWxk
dW5nZW4sIFdhcm51bmdlbiBldGMuIERhcyBQcm9ncmFtbSB3aXJkIG5pY2h0IGdlc3RhcnRldC4=



In [106]:
message = create_message("datathon@lumos-consulting.at", "team4@lumos-consulting.at", "Test1","Taserster")
# message = create_message("team4@lumos-consulting.at", "team4@lumos-consulting.at", "Test_1", "Ich habe WAREHOUSE Produkte im IT&OFFICEKatalog15 firstclass standardmaessig installiert. Die Installation lief fehlerfrei ab und wurde ordnungsgemaess abgeschlossen. Das Programm wurde anschliessend registriert.  Beim Programmaufruf kommt fuer ca. 4-5 Sekunden die Sanduhr auf dem Bildschirm, sonst keine weitere Reaktionen, keine Fehlermeldungen, Warnungen etc. Das Programm wird nicht gestartet.")
message

VGFzZXJzdGVy



{'raw': 'Q29udGVudC1UeXBlOiB0ZXh0L3BsYWluOyBjaGFyc2V0PSJ1dGYtOCIKTUlNRS1WZXJzaW9uOiAxLjAKQ29udGVudC1UcmFuc2Zlci1FbmNvZGluZzogYmFzZTY0CnRvOiB0ZWFtNEBsdW1vcy1jb25zdWx0aW5nLmF0CmZyb206IGRhdGF0aG9uQGx1bW9zLWNvbnN1bHRpbmcuYXQKc3ViamVjdDogVGVzdDEKClZHRnpaWEp6ZEdWeQo='}

In [86]:
send_message(service, "me", message)

Message Id: 177737d99b836960


In [87]:
messages = receive_unread_messages(service)


177737d99b836960
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 10:39:06 -0800
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64
to: team4@lumos-consulting.at
from: team4@lumos-consulting.at
subject: Test_1
Date: Fri, 5 Feb 2021 10:39:06 -0800
Message-Id: <CAEzfA9rqW56A3LSUJOu5KywkeH-G+tWL4sPoZaFsKqabDYL1nw@mail.gmail.com>

SWNoIGhhYmUgV0FSRUhPVVNFIFByb2R1a3RlIGltIElUJk9GRklDRUthdGFsb2cxNSBmaXJzdGNs
YXNzIHN0YW5kYXJkbWFlc3NpZyBpbnN0YWxsaWVydC4gRGllIEluc3RhbGxhdGlvbiBsaWVmIGZl
aGxlcmZyZWkgYWIgdW5kIHd1cmRlIG9yZG51bmdzZ2VtYWVzcyBhYmdlc2NobG9zc2VuLiBEYXMg
UHJvZ3JhbW0gd3VyZGUgYW5zY2hsaWVzc2VuZCByZWdpc3RyaWVydC4gIEJlaW0gUHJvZ3JhbW1h
dWZydWYga29tbXQgZnVlciBjYS4gNC01IFNla3VuZGVuIGRpZSBTYW5kdWhyIGF1ZiBkZW0gQmls
ZHNjaGlybSwgc29uc3Qga2VpbmUgd2VpdGVyZSBSZWFrdGlvbmVuLCBrZWluZSBGZWhsZXJtZWxk
dW5nZW4sIFdhcm51bmdlbiBldGMuIERhcyBQcm9ncmFtbSB3aXJkIG5pY2h0IGdlc3RhcnRldC4=



In [23]:
labellist=np.arange(1,7)
PATH="final_model"
from transformers import AutoModelForSequenceClassification, AutoTokenizer

def predictAbteilung(text):
    model = AutoModelForSequenceClassification.from_pretrained(PATH)
    tokenizer = AutoTokenizer.from_pretrained(PATH)
    labels = torch.tensor([1]).unsqueeze(0)
    inputs = tokenizer(text, return_tensors="pt")
    prediction = model(**inputs, labels=labels)
    logits=prediction.logits.detach().numpy()[0]
    probabilities=softmax(logits)
    return arg3max(probabilities)

In [64]:
def arg3max(liste):
    retList = [(0,0),(0,0),(0,0)]
    print(liste)
    for (count,value) in enumerate(liste):
        if value > retList[0][1]:
            if value > retList[1][1]:
                if value > retList[2][1]:
                    retList[0]=retList[1]
                    retList[1]=retList[2]
                    retList[2]=(count,value)
                else:
                    retList[0]=retList[1]
                    retList[1]=(count,value)
            else:
                retList[0]=(count,value)
    return retList
        
def mail_weiterleitung(service):
    messages = receive_all_messages(service)
    for mid,message in messages:
        text=get_text_from_message(message)
        predictions = predictAbteilung(text)
        abteilung1 = labellist[predictions[2][0]]
        abteilung2 = labellist[predictions[1][0]]
        abteilung3 = labellist[predictions[0][0]]
        text+="\n__________________________________________\nDiese Mail ist mit {p1:.2f} % Wahrscheinlichkeit für Abteilung {a1} bestimmt.\n\nMit {p2:.2f} % Wahrscheinlichkeit könnte Sie für Abteilung {a2} und mit {p3:.2f} % Wahrscheinlichkeit für Abteilung {a3} bestimmt sein.".format(p1=predictions[2][1]*100,p2=predictions[1][1]*100,p3=predictions[0][1]*100,a1=str(abteilung1),a2=str(abteilung2),a3=str(abteilung3))
        if predictions[2][1]-predictions[2][1]>0.2:
            betreff="Abteilung "+str(abteilung1)+":  "+str(message["subject"])
        else:
            betreff="Unklarheit bei Zuweisung: "+str(message["subject"])
        wl_message = create_message("team4@lumos-consulting.at", "team4@lumos-consulting.at", betreff, text)
        send_message(service, "me", wl_message)
        deleteMail(service,mid)

In [65]:
mail_weiterleitung(service)

17773354db0c32bc
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 09:20:08 -0800
Content-Type: text/html; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64
to: team4@lumos-consulting.at
from: team4@lumos-consulting.at
subject: Unklarheit bei Zuweisung: Test_2
Date: Fri, 5 Feb 2021 09:20:08 -0800
Message-Id: <CAEzfA9oHaEya1c9rUBvyXvKSLfkGwUmNvU_tqBEaubhOpG7Qfw@mail.gmail.com>

VTJWb2NpQm5aV1ZvY25SbGN5QlhZWEpsYUc5MWMyVXRWR1ZoYlN3Z2FXTm9JR2hoWW1VZ1pHbGxJ
RU5rSUVKcGJHd2dRMnhsWVc1bApjaUJUZEhWa2FXOGdkbVZ5YTJGMVpuUXNJRzUxYmlCb1lYUWda
R1Z5SUV2RHBIVm1aWElnWldsdUlGQnliMkpzWlcwZ1pYSWdhMkZ1CmJpQnRhWFFnWkdsbGMyVnlJ
Rk52Wm5SM1lYSmxJR3RsYVc1bElIVndaR0YwWlhNZ2JXRmphR1Z1TENCa1lTQmthV1VnVTI5bWRI
ZGgKY21VZ2FXMXRaWElnYm05amFDQmhkV1lnYldWcGJtVnVJRTVoYldWdUlDaEJZMk4xYm5RcElI
SmxaMmx6ZEhKcFpYSjBJR2x6ZEM0ZwpSWE1nYVhOMElHRjFZMmdnYm1samFIUWdiY08yWjJ4cFky
Z2daR2xsYzJWeklGQnliMlIxYTNRZ1lYVnpJRzFsYVc1bGJpQkJZMk52CmRXNTBJSHAxSUd6RHRu
TmphR1Z1TGlCW

Token indices sequence length is longer than the specified maximum sequence length for this model (974 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (974) must match the size of tensor b (512) at non-singleton dimension 1